In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Ustawienia dla wykresów
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

# Ustawienie wyświetlania liczb zmiennoprzecinkowych
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Ustawienie wyświetlania wszystkich kolumn
pd.set_option('display.max_columns', None)


In [ ]:
# Wczytanie danych
df_inflation = pd.read_csv('inflation.tsv', sep='\t')
df_unemployment = pd.read_csv('unemployment.tsv', sep='\t')

print("Dane o inflacji:")
print(f"Wymiary: {df_inflation.shape}")
print("\nPierwsze wiersze danych:")
display(df_inflation.head())

print("\nDane o bezrobociu:")
print(f"Wymiary: {df_unemployment.shape}")
print("\nPierwsze wiersze danych:")
display(df_unemployment.head())


In [ ]:
# Analiza brakujących danych
print("Brakujące dane w zbiorze inflacji:")
print(df_inflation.isnull().sum())
print("\nBrakujące dane w zbiorze bezrobocia:")
print(df_unemployment.isnull().sum())

print("\nPodstawowe statystyki dla inflacji:")
display(df_inflation.describe())
print("\nPodstawowe statystyki dla bezrobocia:")
display(df_unemployment.describe())


In [ ]:
# Funkcje pomocnicze do czyszczenia danych
def clean_numeric_data(value):
    if pd.isna(value):
        return np.nan
    if isinstance(value, str):
        # Usuń znaki specjalne i spacje
        value = value.replace(',', '.').strip()
        try:
            return float(value)
        except ValueError:
            return np.nan
    return value

# Czyszczenie danych
# Zakładamy, że kolumny numeryczne mogą wymagać konwersji
numeric_columns_inflation = df_inflation.select_dtypes(include=['object']).columns
numeric_columns_unemployment = df_unemployment.select_dtypes(include=['object']).columns

for col in numeric_columns_inflation:
    df_inflation[col] = df_inflation[col].apply(clean_numeric_data)

for col in numeric_columns_unemployment:
    df_unemployment[col] = df_unemployment[col].apply(clean_numeric_data)

# Usuwanie wierszy z brakującymi danymi (jeśli występują)
df_inflation = df_inflation.dropna()
df_unemployment = df_unemployment.dropna()

print("Wymiary po czyszczeniu:")
print(f"Inflacja: {df_inflation.shape}")
print(f"Bezrobocie: {df_unemployment.shape}")


In [ ]:
# Wizualizacja danych

# Wykres trendu inflacji
plt.figure(figsize=(15, 6))
plt.title('Trend inflacji w czasie')
plt.plot(df_inflation.index, df_inflation.iloc[:, 1], marker='o')
plt.xlabel('Okres')
plt.ylabel('Inflacja (%)')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

# Wykres trendu bezrobocia
plt.figure(figsize=(15, 6))
plt.title('Trend bezrobocia w czasie')
plt.plot(df_unemployment.index, df_unemployment.iloc[:, 1], marker='o', color='red')
plt.xlabel('Okres')
plt.ylabel('Stopa bezrobocia (%)')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

# Wykres pudełkowy (box plot)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

sns.boxplot(data=df_inflation.iloc[:, 1], ax=ax1)
ax1.set_title('Rozkład inflacji')
ax1.set_ylabel('Inflacja (%)')

sns.boxplot(data=df_unemployment.iloc[:, 1], ax=ax2, color='red')
ax2.set_title('Rozkład bezrobocia')
ax2.set_ylabel('Stopa bezrobocia (%)')

plt.tight_layout()
plt.show()

# Histogram
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

sns.histplot(data=df_inflation.iloc[:, 1], ax=ax1, kde=True)
ax1.set_title('Histogram inflacji')
ax1.set_xlabel('Inflacja (%)')

sns.histplot(data=df_unemployment.iloc[:, 1], ax=ax2, kde=True, color='red')
ax2.set_title('Histogram bezrobocia')
ax2.set_xlabel('Stopa bezrobocia (%)')

plt.tight_layout()
plt.show()


In [ ]:
# Analiza korelacji

# Zakładamy, że mamy te same okresy dla obu zmiennych
# W rzeczywistości może być potrzebne dodatkowe przetwarzanie dat i łączenie danych

# Obliczanie współczynnika korelacji Pearsona
correlation = stats.pearsonr(df_inflation.iloc[:, 1], df_unemployment.iloc[:, 1])
print(f"Współczynnik korelacji Pearsona: {correlation[0]:.3f}")
print(f"Wartość p: {correlation[1]:.3f}")

# Wykres rozproszenia (scatter plot)
plt.figure(figsize=(10, 6))
plt.scatter(df_inflation.iloc[:, 1], df_unemployment.iloc[:, 1], alpha=0.5)
plt.xlabel('Inflacja (%)')
plt.ylabel('Stopa bezrobocia (%)')
plt.title('Zależność między inflacją a bezrobociem')

# Dodanie linii trendu
z = np.polyfit(df_inflation.iloc[:, 1], df_unemployment.iloc[:, 1], 1)
p = np.poly1d(z)
plt.plot(df_inflation.iloc[:, 1], p(df_inflation.iloc[:, 1]), "r--", alpha=0.8)

plt.grid(True)
plt.tight_layout()
plt.show()
